<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/options/binomial_convergence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import norm

# Binomial trees for European options

def callEuropean(S, K, T, sigma, r, q, N=100):
    def f(s):
        s = s if s != 0 else 1.0e-6
        dt = T / N
        up = np.exp(sigma * np.sqrt(dt))
        down = 1 / up
        prob = (np.exp((r - q) * dt) - down) / (up - down)
        discount = np.exp(-r * dt)
        v = np.zeros(N + 1)
        x = s * up ** N
        v[0] = np.maximum(x - K, 0)
        for i in range(1, N + 1):
            x *= down * down
            v[i] = np.maximum(x - K, 0)
        for n in range(N - 1, -1, -1):
            x = s * up ** n
            v[0] = discount * (prob * v[0] + (1 - prob) * v[1])
            for i in range(1, n + 1):
                x *= down * down
                v[i] = discount * (prob * v[i] + (1 - prob) * v[i + 1])
        return v[0]

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)


def putEuropean(S, K, T, sigma, r, q, N=100):
    def f(s):
        s = s if s != 0 else 1.0e-6
        dt = T / N
        up = np.exp(sigma * np.sqrt(dt))
        down = 1 / up
        prob = (np.exp((r - q) * dt) - down) / (up - down)
        discount = np.exp(-r * dt)
        v = np.zeros(N + 1)
        x = s * up ** N
        v[0] = np.maximum(K - x, 0)
        for i in range(1, N + 1):
            x *= down * down
            v[i] = np.maximum(K - x, 0)
        for n in range(N - 1, -1, -1):
            x = s * up ** n
            v[0] = discount * (prob * v[0] + (1 - prob) * v[1])
            for i in range(1, n + 1):
                x *= down * down
                v[i] = discount * (prob * v[i] + (1 - prob) * v[i + 1])
        return v[0]

    if isinstance(S, list) or isinstance(S, np.ndarray):
        return np.array([f(s) for s in S])
    else:
        return f(S)

In [ ]:
# example parameters

S = 50
K = 50
T = 1
sigma = 0.4
r = 0.02
q = 0.03

# preliminary calculations

d1 = (np.log(S/K) + (r-q+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
d2 = d1 - sigma*np.sqrt(T)
qT = np.exp(-q*T)
rT = np.exp(-r*T)

# Call premium (Black-Scholes Value)
N1 = norm.cdf(d1)
N2 = norm.cdf(d2)
C = qT * S * N1 - rT * S * N2
# Put premium (Black-Scholes Value)
N1 = norm.cdf(-d1)
N2 = norm.cdf(-d2)
P = rT * K * N2 - qT * S * N1

N = [10, 20, 40, 80] # example number of interation
print('The Black-Scholes value of the call premium is', C.round(2), end='.\n')
for n in N:
    C_sim = callEuropean(S, K, T, sigma, r, q, n)
    print('When the number of time steps in the binomial model is', n, end=', ')
    print('the simulated value is', C_sim.round(2), 'and the difference with Black-Sholes value is', (C_sim - C).round(2), end='.\n')



The Black-Scholes value of the call premium is 7.49.
When the number of time steps in the binomial model is 10, the simulated value is 7.3 and the difference with Black-Sholes value is -0.19.
When the number of time steps in the binomial model is 20, the simulated value is 7.39 and the difference with Black-Sholes value is -0.1.
When the number of time steps in the binomial model is 40, the simulated value is 7.44 and the difference with Black-Sholes value is -0.05.
When the number of time steps in the binomial model is 80, the simulated value is 7.46 and the difference with Black-Sholes value is -0.02.
